# Arbres : solutionnaire

_Note:  
Ce solutionnaire utilise le **jeu de données de Montréal-Python** car les données sources de la Ville de Montréal peuvent changer dans le temps (structure et données). Le jeu de données de Montréal-Python assure la pérennité du solutionnaire (pas besoin de changer le nom des colonnes, par exemple :) )._

## Installer pandas

`pandas` n'est pas dans la _Standard Library_ (ne vient pas par défaut avec Python), faut donc l'installer
* Pour Colab, c'est installé par défaut, pas besoin d'installer, sinon on pourrait le faire avec `!pip install pandas`
* Machine locale `pip install pandas`

## Charger vos données

### Import

Par convention, on renomme `pd`.

In [ ]:
import pandas as pd

### Charger

Pandas a des readers pour différents formats de données, incluant CSV.

In [ ]:
url = "https://raw.githubusercontent.com/mtlpy/mtlpy-exercises/main/mtl/arbres/arbres-publics.csv"
data = pd.read_csv(url)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Explorer et préparer les données


### Vue d'ensemble

In [ ]:
# volumétrie des colonnes et rangées et
# aperçu des types et valeurs des données
data

,INV_TYPE,EMP_NO,ARROND,ARROND_NOM,Rue,COTE,No_civique,Emplacement,Coord_X,Coord_Y,SIGLE,Essence_latin,Essence_fr,ESSENCE_ANG,DHP,Date_releve,Date_plantation,LOCALISATION,CODE_PARC,NOM_PARC,Longitude,Latitude
0,H,6,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287967.933,5043937.611,GLTRSK,Gleditsia triacanthos 'Skyline',Févier Skyline,Skyline Honey-Locust,25.0,2018-06-26T00:00:00,2004-06-10T00:00:00,NaN,0005-000,RAIMBAULT,-73.715515,45.535151
1,H,7,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287961.715,5043945.204,GLTRSK,Gleditsia triacanthos 'Skyline',Févier Skyline,Skyline Honey-Locust,32.0,2018-06-27T00:00:00,2004-06-18T00:00:00,NaN,0005-000,RAIMBAULT,-73.715595,45.535219
2,H,8,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287956.685,5043932.522,GYDI,Gymnocladus dioicus,Chicot du Canada,Kentucky Coffee Tree,21.0,2018-06-27T00:00:00,2004-06-10T00:00:00,NaN,0005-000,RAIMBAULT,-73.715659,45.535105
3,H,9,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287985.011,5043989.384,FRNIFG,Fraxinus nigra 'Fall Gold',Frêne noir Fall Gold,Fall Gold Black Ash,5.0,2018-07-12T00:00:00,2004-06-18T00:00:00,NaN,0005-000,RAIMBAULT,-73.715298,45.535617
4,H,10,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287961.737,5043987.310,QURU,Quercus rubra,Chêne rouge,Red Oak,34.0,2018-06-27T00:00:00,2004-06-18T00:00:00,NaN,0005-000,RAIMBAULT,-73.715596,45.535598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321194,R,40177,33,Montréal-Est,Boulevard Henri-Bourassa Est ...,S,9150.0,PaRterre,300094.284,5055478.049,ULAM,Ulmus americana,Orme d'Amérique,White Elm,14.0,1990-07-01T00:00:00,NaN,378 m. à l'Est du 9150,NaN,NaN,-73.560363,45.639183
321195,R,40183,33,Montréal-Est,Boulevard Henri-Bourassa Est ...,S,9150.0,PaRterre,300090.780,5055478.048,QUMA,Quercus macrocarpa,Chêne à gros fruits,Bur Oak,14.0,1990-07-01T00:00:00,NaN,343 m. à l'Est du 9150,NaN,NaN,-73.560408,45.639183
321196,R,40185,33,Montréal-Est,Boulevard Henri-Bourassa Est ...,S,9150.0,PaRterre,300096.781,5055478.045,ULAM,Ulmus americana,Orme d'Amérique,White Elm,12.0,1990-07-01T00:00:00,NaN,403 m. à l'Est du 9150,NaN,NaN,-73.560331,45.639183
321197,R,40186,33,Montréal-Est,Boulevard Henri-Bourassa Est ...,S,9150.0,PaRterre,300096.982,5055478.046,ULAM,Ulmus americana,Orme d'Amérique,White Elm,10.0,1990-07-01T00:00:00,NaN,405 m. à l'Est du 9150,NaN,NaN,-73.560328,45.639183


Chaque rangée représente un arbre. Il est légitime de croire qu'il n'y a pas de doublon et de se fier à l'effort d'inventaire mais nous pourrions vouloir analyser les données pour en être sûr (exemple, les latitude et longitude de chaque murale sont biens uniques?).

In [ ]:
# nom des colonnes
data.columns

Index(['INV_TYPE', 'EMP_NO', 'ARROND', 'ARROND_NOM', 'Rue', 'COTE',
       'No_civique', 'Emplacement', 'Coord_X', 'Coord_Y', 'SIGLE',
       'Essence_latin', 'Essence_fr', 'ESSENCE_ANG', 'DHP', 'Date_releve',
       'Date_plantation', 'LOCALISATION', 'CODE_PARC', 'NOM_PARC', 'Longitude',
       'Latitude'],
      dtype='object')

### Vérifier les valeurs

In [ ]:
# arrondissement: 16 valeurs uniques sans erreur apparente
len(data['ARROND_NOM'].unique())
data['ARROND_NOM'].unique()

array(['Ahuntsic - Cartierville',
       'Villeray-Saint-Michel - Parc-Extension',
       'Rosemont - La Petite-Patrie', 'Mercier - Hochelaga-Maisonneuve',
       'Le Plateau-Mont-Royal', 'Ville-Marie',
       'Côte-des-Neiges - Notre-Dame-de-Grâce', 'Le Sud-Ouest',
       'Rivière-des-Prairies - Pointe-aux-Trembles', 'Saint-Léonard',
       'LaSalle', 'Verdun', 'Pierrefonds - Roxboro', 'Saint-Laurent',
       'Anjou', 'Montréal-Est'], dtype=object)

In [ ]:
len(data['Essence_fr'].unique())
# 699 essences en français

len(data['Essence_latin'].unique())
# 703 essences en latin

len(data['ESSENCE_ANG'].unique())
# 696 essences en anglais

len(data['SIGLE'].unique())
# 703 essences par genre/espece/cultivar

703

Bon, on voit que
* on a un bon volume de données
* on a des minuscules, majuscules dans les champs `essence_*`
* on n'a pas la même volumétrie d'essence en fr et en lat

On va devoir faire des requêtes pour explorer les données.

### Sous-ensemble

On peut sélectionner un sous-ensemble de données un peu à la manière de requêtes SQL grâce à `query()` du DataFrame (direct sur le jeu de données, pas sur une colonne ni sur une rangée).

Aussi, les colonnes ayant des valeurs de type strings donnent accès à des fonctions derrière `.str`
* https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html

Pour introspecter les opérations possibles, stockez une colonne dans une variable et introspectez
```
colonne = data['essence_fr']
colonne.str. (+ tab)
```

In [ ]:
# requête (query) sur le jeu de données pour savoir si essence_fr en minuscule contient 'bouleau'
bouleaux = data.query("Essence_fr.str.lower().str.contains('bouleau')", engine="python")
len(bouleaux)

# essence_fr.str.lower() retourne une colonne avec valeurs en string tout en minuscule
# sur cette colonne, aussi en string, on peut utiliser encore `.str` pour d'autres opérations sur string
# on peut donc tester si la chaîne en minuscule contient 'bouleau'
# (note engine="python" pas nécessaire en local, nécessaire pour .ipynb de Colab car default engine est numexpr: merci Ivan!)

1973

## Analyser les données

### Exercice: arrondissement avec le moins de bouleaux

In [ ]:
vc = bouleaux['ARROND_NOM'].value_counts()
vc

Rivière-des-Prairies - Pointe-aux-Trembles    286
Rosemont - La Petite-Patrie                   257
Côte-des-Neiges - Notre-Dame-de-Grâce         250
Pierrefonds - Roxboro                         224
Ahuntsic - Cartierville                       214
LaSalle                                       193
Mercier - Hochelaga-Maisonneuve               147
Le Sud-Ouest                                  109
Verdun                                        109
Villeray-Saint-Michel - Parc-Extension         56
Saint-Laurent                                  54
Saint-Léonard                                  35
Le Plateau-Mont-Royal                          27
Ville-Marie                                    12
Name: ARROND_NOM, dtype: int64

In [ ]:
vc.idxmin()

'Ville-Marie'

## Aller plus loin

### Exercice: plus d'occurences en premier

On l'a par défaut avec `value_counts()`... essayons alors de faire l'inverse? Moins d'occurences en premier?

Une introspection montre rapidement qu'on peut trier (_sort_) sur l'index ou sur les valeurs... ben voilà!

```
vc.
vc.sort_values?
```

In [ ]:
vc.sort_values()

Ville-Marie                                    12
Le Plateau-Mont-Royal                          27
Saint-Léonard                                  35
Saint-Laurent                                  54
Villeray-Saint-Michel - Parc-Extension         56
Le Sud-Ouest                                  109
Verdun                                        109
Mercier - Hochelaga-Maisonneuve               147
LaSalle                                       193
Ahuntsic - Cartierville                       214
Pierrefonds - Roxboro                         224
Côte-des-Neiges - Notre-Dame-de-Grâce         250
Rosemont - La Petite-Patrie                   257
Rivière-des-Prairies - Pointe-aux-Trembles    286
Name: ARROND_NOM, dtype: int64

### Exercice: 3 premiers résultats dans un format agréable à lire

Le format est déjà agréable à lire...

Mais prenons les 3 premiers résultats seulement.

On doit pouvoir slicer, non?

In [ ]:
vc.iloc[0:3]

Rivière-des-Prairies - Pointe-aux-Trembles    286
Rosemont - La Petite-Patrie                   257
Côte-des-Neiges - Notre-Dame-de-Grâce         250
Name: ARROND_NOM, dtype: int64

### Exercice bonus: trier sur nom d'arrondissement

On l'a dit... on peut faire _sort_ sur index ou valeurs...

```
vc.sort_index?
```

In [ ]:
vc.sort_index()

Ahuntsic - Cartierville                       214
Côte-des-Neiges - Notre-Dame-de-Grâce         250
LaSalle                                       193
Le Plateau-Mont-Royal                          27
Le Sud-Ouest                                  109
Mercier - Hochelaga-Maisonneuve               147
Pierrefonds - Roxboro                         224
Rivière-des-Prairies - Pointe-aux-Trembles    286
Rosemont - La Petite-Patrie                   257
Saint-Laurent                                  54
Saint-Léonard                                  35
Verdun                                        109
Ville-Marie                                    12
Villeray-Saint-Michel - Parc-Extension         56
Name: ARROND_NOM, dtype: int64

# Cheat sheet

In [ ]:
import pandas as pd

# data import
url = "https://raw.githubusercontent.com/mtlpy/mtlpy-exercises/main/mtl/arbres/arbres-publics.csv"
data = pd.read_csv(url)

# data exploration
data
data.columns
data['ARROND_NOM'].unique()

# data preparation
bouleaux = data.query("Essence_fr.str.lower().str.contains('bouleau')", engine="python")

# data analysis
vc = bouleaux['ARROND_NOM'].value_counts()
vc.idxmin()

# more
vc.sort_values()
vc.iloc[0:3]
vc.sort_index()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Ahuntsic - Cartierville                       214
Côte-des-Neiges - Notre-Dame-de-Grâce         250
LaSalle                                       193
Le Plateau-Mont-Royal                          27
Le Sud-Ouest                                  109
Mercier - Hochelaga-Maisonneuve               147
Pierrefonds - Roxboro                         224
Rivière-des-Prairies - Pointe-aux-Trembles    286
Rosemont - La Petite-Patrie                   257
Saint-Laurent                                  54
Saint-Léonard                                  35
Verdun                                        109
Ville-Marie                                    12
Villeray-Saint-Michel - Parc-Extension         56
Name: ARROND_NOM, dtype: int64

# Licence

Copyright 2021 Montréal-Python

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
